# Authors: Kai Koh, Blue Le
# Student ID: 64276083, 41086772
# Group: Data Freaks

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.2/spark-2.3.2-bin-hadoop2.7.tgz
!tar xf spark-2.3.2-bin-hadoop2.7.tgz
!pip install -q pyspark
!pip install LocalitySensitiveHashing
!pip install BitVector
!pip install gdelt


     |████████████████████████████████| 215.6MB 117kB/s 
     |████████████████████████████████| 204kB 42.9MB/s 
     |████████████████████████████████| 71kB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/63/8b/25/5fca539d61443cdc7506d849e685aa3e5fa2d43f40d8515a65
Successfully built LocalitySensitiveHashing
     |████████████████████████████████| 133kB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/75/a0/12/73dcb6e70a9d650ac2253472497f78ac874709a7c39a415f1a
Successfully built BitVector
     |████████████████████████████████| 778kB 2.8MB/s 


In [0]:
import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

In [0]:
"""
Compilation of words related to Marvel's Cinematic Universe movies, which will be used
to obtain relevant GDELT data, as well as to calculate Jaccard similarity.
"""
must_contain = list(set(
    [
        "marvel", "marvel's", "mcu", 
        "marvel cinematic universe",
        "marvel's cinematic universe",
        "marvels cinematic universe"
    ]
))

phase_one = list(set(
    [
        "phil coulson", "clark gregg", "christine everhart", "leslie bibb",
        "samuel l. jackson", "samuel jackson", "nuck fury", "harold hogan", 
        "happy hogan", "jon favreau", "JARVIS", "j.a.r.v.i.s", "paul bettany",
        "virginia potts", "pepper potts", "gwyneth paltrow", "raza", 
        "faran tahir", "james rhodes", "rhodey", "terrance howard", 
        "don cheadle", "obadiah stane", "jeff bridges", "howard stark", 
        "gerard sanders", "john slattery", "dominic cooper", "tony stark",
        "iron man", "robert downey jr.", "ho yinsen", "shaun toub", 
        "bruce banner", "hulk", "edward norton", "lou ferrigno", "mark ruffalo",
        "emil blonsky", "tim roth", "roger harrington", "martin starr", 
        "betty ross", "liv tyler", "thaddeus ross", "thunderbolt", "william hurt",
        "leonard samson", "ty burrell", "samuel sterns", "tim blake nelson",
        "justin hammer", "sam rockwell", "peter parker", "max favreau",
        "chess roberts", "olivia munn", "natasha romanoff", "black widow",
        "scarlett johansson", "stern", "garry shandling", "anton vanko", 
        "yevgeni lazarev", "ivan vanko", "mickey rourke", "clint barton",
        "hawkeye", "jeremy renner", "fandral", "josh dallas", "jane foster",
        "natalie portman", "frigga", "rene russo", "heimdall", "idris elba",
        "hogun", "tadanobu asano", "laufey", "colm feore", "darcy", "darcy lewis",
        "kat dennings", "loki", "tom hiddleston", "odin", "anthony hopkins",
        "erik selvig", "stellan skarsgard", "sif", "lady sif", "jaimie alexander",
        "jasper sitwell", "maximiliano hernandez", "thor", "chris hemsworth",
        "volstagg", "ray stevenson", "james barnes", "james buchanan barnes", 
        "bucky", "sebastian stan", "brandt", "michael brandon", "peggy carter",
        "hayley atwell", "jacques dernier", "bruno ricci", "timothy dugan",
        "dum dum", "neal mcdonough", "abraham erskine", "stanly tucci", 
        "james montgomery falsworth", "james falsworth", "jj feild", "gilmore hodge",
        "lex shrapnel", "gabe jones", "derek luke", "heinz kruger", "richard armitage",
        "jim morita", "kenneth choi", "chester phillips", "tommy lee jones",
        "steve rogers", "captain america", "chris evans", "johann schmidt",
        "red skull", "hydra", "s.h.i.e.l.d", "hugo weaving", "arnim zola",
        "toby jones", "avengers" "hawley", "jenny agutter", "maria hill",
        "cobie smulders", "georgi luchkov", "jerzy skolimowski", "gideon malick",
        "powers boothe", "thanos", "damion poitier", "the other", "alexis denisof",
        "jon favreau", "mark fergus", "hawk ostby", "art marcum", "matt holloway",
        "avi arad", "kevin feige", "louis leterrier", "the incredible hulk",
        "zak penn", "gale anne hurd", "iron man 2", "the first avenger",
        "justin theroux", "god of thunder", "joe johnston", "christopher markus",
        "stephen mcfeely", "the avengers", "avengers", "joss whedon", "betty ross",
        "asgard", "tesseract", "phase one", "phase 1"
    ]
))

phase_two = list(set(
    [
        "iron man 3", "shane black", "drew pearce", "the dark world", "dark elves",
        "alan taylor", "christopher l. yost", "christopher markus", "stephen mcfeely",
        "the winter soldier", "anthony russo", "joe russo", "russo brothers",
        "kevin feige", "guardians of the galaxy", "james gunn", "nicole perlmann",
        "age of ultron", "joss whedon", "ant-man", "peyton reed", "edgar wright",
        "joe cornish", "adam mckay", "paul rudd", "the mandarin", "josh brolin",
        "ellen brandt", "stephanie szostak", "matthew ellis", "william sadler",
        "maya hansen", "rebecca hall", "harley keener", "ty simpkins", 
        "aldrich killian", "guy pearce", "rodriguez", "miguel ferrer", "eric savin",
        "james badge dale", "trevor slattery", "ben kingsley", "jack taggart",
        "ashley hamilton", "wu", "wang xueqi", "algrim", "kurse", 
        "adewale akinnuoye-agbaje", "bor", "tony curran", "ian boothby", 
        "jonathan howard", "carina", "ophelia lovibond", "eir", "alice krige",
        "malekith", "christopher eccleston", "richard", "chris o'dowd", "the collector",
        "taneleer tivan", "benicio del toro", "tyr", "clive russell", "georges batroc",
        "georges st-pierre", "sharon carter", "agent 13", "emily vancamp", "list",
        "henry goodman", "cameron klein", "aaron himelstein", "pietro maximoff",
        "quicksilver", "aaron taylor-johnson", "wanda maximoff", "scarlet witch",
        "elizabeth olsen", "alexander pierce", "robert redford", "jack rollins",
        "callan mulvey", "brock rumlow", "frank grillo", "wolgang von strucker",
        "strucker", "thomas kretschmann", "sam wilson", "falcon", "anthony mackie",
        "bereet", "melia kreiling", "the broker", "christopher fairbank", "cosmo",
        "fred", "rhomann dey", "john c. reilly", "drax", "drax the destroyer",
        "the destroyer", "dave bautista", "gamora", "zoe saldana", "groot", "vin diesel",
        "howard the duck", "seth green", "korath", "djimon hounsou", "kraglin",
        "sean gunn", "nebula", "karen gillan", "merdith quill", "laura haddock",
        "grandpa quill", "gregg henry", "peter quill", "star lord", "chris pratt",
        "irani rael", "nova prime", "glenn close", "rocket", "bradley cooper",
        "ronan the accuser", "ronan", "lee pace", "garthan saal", "peter serafinowicz",
        "yondu udonta", "yondu", "michael rooker", "laura barton", "linda cardellini",
        "helen cho", "claudia kim", "f.r.i.d.a.y.", "veronica", "kerry condon",
        "hulkbuster", "ulysses klaue", "andy serkis", "ultron", "james spader",
        "vision", "paul bettany", "mitchell carson", "martin donovan", "cassie",
        "darren cross", "yellowjacket", "yellow jacket", "corey stoll", "dale",
        "dave", "gregg turkington", "tip harris", "gale", "wood harris", "kurt",
        "david dastmalchian", "scott lang", "paul rudd", "luis", "micahel pena",
        "maggie", "judy green", "paxton", "hank pym" "pym", "bobby cannavale",
        "michael douglas", "hope van dyne", "janet van dyne", "wasp", "hayley lovitt",
        "evangeline lilly", "phase 2", "phase two"
        
    ]
))

phase_three = list(set(
    [
        "infinity stones", "mind stone", "soul stone", "power stone",
        "space stone", "time stone", "reality stone", "vormir", "civil war",
        "doctor strange", "doctor steven strange", "guardians of the galaxy vol. 2",
        "homecoming", "spider-man", "spider man", "ragnarok", "black panther",
        "infinity war", "ant-man and the wasp", "captain marvel", "endgame", 
        "far from home", "zachary levi", "ayo", "florence kasumba", "t'chaka",
        "john kani", "t'challa", "black panther", "chadwick boseman", "may parker",
        "marisa tomei", "peter parker", "tom holland", "everet ross", "everett k. ross",
        "martin freeman", "miriam sharpe", "alfre woodard", "helmu zemo", "daniel bruhl",
        "ancient one", "tilda swinton", "dormammu", "benedict cumberbatch",
        "daniel drumm", "mark anthony brighton", "hamir", "topo wresniwiro", "kaecilius",
        "mads mikkelsen", "lucian aster", "scott adkins", "tina minoru", "linda louise duan",
        "karl mordo", "chiwetel ejiofor", "christine palmer", "rachel mcadams",
        "jonathan pangborn", "benjamin bratt", "stephen strange", "nicodemus west",
        "michael stuhlbarg", "wong", "benedict wong", "ayesha", "elizabeth debicki",
        "brahl", "stephen blackehart", "charlie-27", "ving rhames", "ego", "kurt russell",
        "gef the ravager", "steve agee", "half-nut", "jimmy urine", "mainframe",
        "miley cyrus", "mantis", "pom klementieff", "martinex", "michael rosenbaum",
        "narblik", "terence rosemore", "oblo", "joe fria", "aleta ogord", "michelle yeoh",
        "stakar ogord", "sylvester stallone", "retch", "evan jones", "scrote",
        "mike escamilla", "taserface", "chris sullivan", "tullk", "tommy flanagan",
        "abraham", "abraham attah", "betty brant", "angourie rice", "jackson brice",
        "shocker", "logan marshall-green", "charles", "micahel barbieri", "cindy",
        "tiffany espensen", "mr. cobbwell", "tunde adebimpe", "aaron davis", 
        "donald glover", "mr. delmar", "hemky madera", "mac gargan", "michael mando",
        "mr. hapgood", "john penick", "anne marie hoag", "tyne daly", "jason lonello",
        "jorge lendeborg jr.", "michelle jones", "zendaya", "karen", "jennifer connelly",
        "liz", "laura harrier", "phineas mason", "tinkerer", "michael chernus",
        "principal morita", "kenneth choi", "ned", "jacob batalon", "sally", 
        "isabella amara", "herman schultz", "shocker", "bokeem woodbine",
        "seymour", "j. j. totah", "eugene thompson", "tony revolori", "tiny",
        "ethan dizon", "adrian toomes", "vulture", "michael keaton", "doris toomes",
        "garcelle beauvais", "ms. warren", "selenis leyva", "coach wilson",
        "grandmaster", "jeff goldblum", "hela", "cate blanchett", "korg", "taika waititi",
        "skurge", "karl urban", "surtur", "clancy brown", "topaz", "rachel house",
        "valkyrie", "tessa thompson", "linda", "nabiyah be", "m'baku", "winston duke",
        "n'jadaka", "erki stevens", "killmonger", "michael b. jordan", "n'jobu",
        "sterling k. brown", "nakia", "lupita nyong'o", "okoye", "danai gurira",
        "ramonda", "angela bassett", "shuri", "letitia wright", "w'kabi", 
        "daniel kaluuya", "xoliswa", "sydelle noel", "zuri", "forest whitaker",
        "eitri", "peter dinklage", "corvus glaive", "michael james shaw",
        "ebony maw", "tom vaughan-lawlor", "proxima midnight", "carrie coon",
        "monique ganderton", "cull obsidian", "terry notary", "sonny burch", "walton goggins",
        "bill foster", "laurence fishburne", "ava starr", "ghost", "hannah john-kamen",
        "elihas starr", "michael cerveris", "uzman", "divian ladwa", "jimmy woo",
        "randall park", "att-lass", "algenis perez soto", "bron-char", "rune temte",
        "carol denvers", "captain marvel", "brie larson", "mar-vell", "wendy lawson",
        "annette bening", "minn-erva", "gemma chan", "maria rambeau", "lashana lynch",
        "monica rambeau", "akrira akbar", "supreme intelligence", "talos", "ben mendelsohn",
        "yon-rogg", "jude law", "akihiko", "hiroyuki sanada", "morgan stark", 
        "alexandra rachael rabe", "scott derrickson", "jon spaihts", "c. robert cargill",
        "james gunn", "jon watts", "jonathan goldstein", "john francis daley", "christopher ford",
        "chris mckenna", "erik sommers", "amy pascal", "taika waititi", "eric pearson",
        "craig kyle", "christopher l. yost", "ryan coogler", "joe robert cole",
        "christpher markus", "peyton reed", "andrew barrer", "gabriel ferrari",
        "stephen broussard", "anna boden", "ryan fleck", "geneva robertson-dworet",
        "phase 3", "phase three"
    ]
))

total_bag = list(set(must_contain + phase_one + phase_two + phase_three))
print(total_bag)

['linda cardellini', 'hugo weaving', 'edgar wright', 'brie larson', 'wendy lawson', 'far from home', 'ian boothby', 'iron man 3', 'iron man', 'nova prime', "t'chaka", 'clancy brown', 'adrian toomes', 'james falsworth', 'miley cyrus', 'jason lonello', 'luis', 'stephen broussard', 'ant-man and the wasp', 'phase one', 'howard the duck', 'stern', 'aaron davis', 'josh brolin', 'christopher l. yost', 'topaz', 'supreme intelligence', 'ragnarok', 'maria rambeau', 'wolgang von strucker', 'emily vancamp', 'gamora', 'gabe jones', 'korath', 'virginia potts', 'richard armitage', 'martin starr', 'bruce banner', 'algrim', 'tommy flanagan', 'ebony maw', 'nabiyah be', "m'baku", 'claudia kim', 'ethan dizon', 'christine everhart', 'loki', 'the broker', 'mantis', 'christine palmer', 'abraham attah', 'morgan stark', 'james buchanan barnes', 'hiroyuki sanada', 'benedict cumberbatch', 'anthony mackie', 'stephen strange', 'sylvester stallone', 'ashley hamilton', 'josh dallas', 'martinex', 'herman schultz', 'p

In [0]:
"""
Helper functions referenced from the labs
"""
def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(a, list(b) if isinstance(b, pyspark.resultiterable.ResultIterable) else b) for a, b in x.take(100)])
  else:
    print(x)

def load_RDD(textfile, total_bag):
  """Loads and Processes RDDs into list of (word, count) tuples"""
  
  file = sc.textFile(textfile)
  # split each line into (word, 1) tuples
  words = file.flatMap(lambda line: [(line.lower(), 1) for word in total_bag if word in line])
  # reduce by key (the word) the counts and sort descending
  counts = words.reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False) 
  
  return counts

In [0]:
"""
Jaccard Similarity Computation
"""
def jaccard_similiarity(list1, list2):
    common = len(list(set(list1).intersection(list2)))
    combine = (len(list1) + len(list2)) - common
    return float(common / combine)

In [0]:
"""
Retrieving documents from GDELT 2.0 GKG and writing them
into a file - documents.txt -.

NOTE: Please only run this once as it is very time consuming!
"""

import gdelt

def retrieve_data():
  gd2 = gdelt.gdelt(version=2)

  y = '9'
  m = '04 05'.split(' ')
  d = '01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31'.split(' ')

  mi = 0
  di = 0
  i = 0

  string = '201{} {} {}'.format(y, m[mi], d[di])
  end = '2019 05 01'

  while string != '2019 05 01':
    try:
      events = gd2.Search([string], table='gkg', output='csv', coverage=False)
    except ValueError:
      print('Incorrect date provided or server error') # no data obtained
    else:
      file = open('documents.txt', 'a+')
      file.write(events)
      file.close()
      print('Data from', string, 'successfully written to file')

    if di != len(d) - 1:
      di += 1
    else:
      di = 0
      if mi != len(m) - 1:
        mi += 1
      else:
        mi = 0

    string = '201{} {} {}'.format(y, m[mi], d[di])


retrieve_data()



Data from 2019 04 01 successfully written to file
Data from 2019 04 02 successfully written to file
Data from 2019 04 03 successfully written to file
Data from 2019 04 04 successfully written to file
Data from 2019 04 05 successfully written to file
Data from 2019 04 06 successfully written to file
Data from 2019 04 07 successfully written to file
Data from 2019 04 08 successfully written to file
Data from 2019 04 09 successfully written to file
Data from 2019 04 10 successfully written to file
Data from 2019 04 11 successfully written to file
Data from 2019 04 12 successfully written to file
Data from 2019 04 13 successfully written to file
Data from 2019 04 14 successfully written to file
Data from 2019 04 15 successfully written to file
Data from 2019 04 16 successfully written to file


/usr/local/lib/python3.6/dist-packages/gdelt/parallel.py:103: UserWarning: GDELT did not return data for date time 20190417234500
  warnings.warn(message)


Incorrect date provided or server error
Data from 2019 04 18 successfully written to file
Data from 2019 04 19 successfully written to file
Data from 2019 04 20 successfully written to file
Data from 2019 04 21 successfully written to file
Data from 2019 04 22 successfully written to file
Data from 2019 04 23 successfully written to file
Data from 2019 04 24 successfully written to file
Data from 2019 04 25 successfully written to file
Data from 2019 04 26 successfully written to file
Data from 2019 04 27 successfully written to file


/usr/local/lib/python3.6/dist-packages/gdelt/parallel.py:103: UserWarning: GDELT did not return data for date time 20190428234500
  warnings.warn(message)


Incorrect date provided or server error
Data from 2019 04 29 successfully written to file
Data from 2019 04 30 successfully written to file
Incorrect date provided or server error


In [0]:
"""
Module to sieve out documents that contain words related to 
the Marvel dictionary of words
"""

import re

def filter_documents(filename):
  store = []

  file = open(filename)
  line = file.readline()
  count = 0
  while line:
    line = file.readline()
    data = re.sub("[^a-zA-Z0-9\s]", ',', line.lower()).split(',')
    for content in data:
      for word in must_contain:
        if len(word) > 1:
          if content == word:
            if data not in store:
              store.append(data)
        else:
          if content in word:
            if data not in store:
              store.append(data)
    count += 1
  return store

store = filter_documents('documents.txt')

In [0]:
#show top 50
count = 0
while count < 50:
  print(store[count])
  count += 1

['162', '20190401234500', '162', '20190401234500', '1', 'iheart', 'com', 'https', '', '', 'froggy999', 'iheart', 'com', 'content', '2019', '04', '01', 'april', '2019', 'movie', 'preview', 'avengers', 'endgame', 'pet', 'sematary', 'more', '', '', '', 'tax', 'fncact', 'tax', 'fncact', 'fool', 'crisislex', 'crisislexrec', 'ungp', 'forests', 'rivers', 'oceans', 'tax', 'fncact', 'king', '', '', 'ungp', 'forests', 'rivers', 'oceans', '315', 'crisislex', 'crisislexrec', '143', 'tax', 'fncact', 'king', '331', 'tax', 'fncact', 'fool', '13', '', '', '', '', 'stephen king', '', 'stephen king', '331', '', '', '', '', '', '0', '8', '3', '2', '4', '7', '2', '16', '8', '2', '4', '115', '', '', '', 'wc', '115', 'c1', '1', '1', 'c12', '1', '7', 'c12', '10', '7', 'c12', '12', '3', 'c12', '13', '1', 'c12', '14', '3', 'c12', '3', '4', 'c12', '5', '3', 'c12', '7', '6', 'c12', '8', '1', 'c12', '9', '5', 'c13', '12', '1', 'c13', '4', '1', 'c14', '1', '6', 'c14', '10', '3', 'c14', '11', '15', 'c14', '2', '7',

In [0]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
import pyspark.sql.functions as f

sqlContext = SQLContext(sc)
data = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('documents.txt')
drop_list = ['_c0', 'DATE', 'SourceCollectionIdentifier', 'SourceCommonName', 'Counts', 'V2Counts', 'Locations', 'V2Locations', 'V2Tone', 'Dates', 'GCAM', 'SharingImage', 'RelatedImages', 'SocialImageEmbeds', 'SocialVideoEmbeds', 'Amounts', 'TranslationInfo', 'Extras']

data = data.select([column for column in data.columns if column not in drop_list])

data.show(30)


+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      GKGRECORDID|  DocumentIdentifier|              Themes|            V2Themes|             Persons|           V2Persons|       Organizations|     V2Organizations|          Quotations|            AllNames|
+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20190401234500-0|https://www.stett...|TAX_FNCACT;TAX_FN...|BAN,2056;GENERAL_...|gerald butts;just...|Gerald Butts,1749...|parliament hill;t...|Parliament Hill,4...|                null|Pierre Poilievre,...|
| 20190401234500-1|https://www.krem....|TAX_FNCACT;TAX_FN...|BAN,130;BAN,639;B...|                null|                null|house committee o...|House Committee O..

In [0]:
%%writefile minhash.py

from pyspark.mllib.linalg import SparseVector
import numpy as np

def min_hash(vec_obj, seed_one, seed_two, seed_big, num_buckets):
    """ Finds out the type of vector input: python list, numpy array or sparse vector, and
    calculates the minimum hash of the vector using 4 seeds: positive integer, positive
    integer (including 0), integer greater than total num of elements in vector and num
    of buckets. Then maps the indices to the hash values and obtain the minimum hash thereon.
    The minimum hash value is in the range (0, num_buckets).
    Steps - 
        1: Multiplies the index by the non-zero seed_one.
        2: Adds the bias seed_two (can be 0).
        3: Modulo seed_big, a number larger than the number of elements.
        4: Modulo num_buckets, the number of buckets.
    """

    indices = None
    if type(vec_obj) is list or type(vec_obj) is np.ndarray:
        indices = np.arange(len(vec_obj), dtype=np.int)
    elif type(vec_obj) is SparseVector:
        indices = vec_obj.indices
    else:
        raise Exception("Array type {} is invalid!".format(type(vec_obj)))

    return np.array(((indices * seed_one + seed_two) % seed_big) % num_buckets).min()


Writing minhash.py


In [0]:
%%writefile lsh.py

import minhash
import functools
import numpy as np
import scipy.spatial.distance as distance

from pyspark.mllib.linalg import SparseVector

class LSHModel:

    def __init__(self, signatures, bands, vec_buck, buck_vec, buckets, jaccard_point):
        self.signatures = signatures
        self.bands = bands
        self.vectors_buckets = vec_buck
        self.buckets_vectors = buck_vec
        self.buckets = buckets
        self.scores = jaccard_point

def distance_vector(bucket_vectors):
    """
    Generates a pairwise, summed Jaccard distance metric for all the elements
    in a cluster / bucket. Input bucket_vectors pairs are of the form:
    bucket ID, list of vectors. Outputs bucket ID, jaccard distance.
    """
    bid, X = bucket_vectors[0], bucket_vectors[1].data
    if type(X[0]) is SparseVector:
        Y = np.array([x.toArray() for x in X])
        X = Y
    return [bid, distance.pdist(np.array(X), 'jaccard').sum()]

def run(data, large_num, bin_num, n, bands, cluster):
    """
    Starts LSH
    """
    zipped = data.zipWithIndex()
    seeds = np.vstack([np.random.random_integers(large_num, size = n), np.random.random_integers(0, large_num, size = n)]).T
    hashes = [functools.partial(hasher.min_hash, a = s[0], bands = s[1], large_num = large_num, bin_num = bin_num) for s in seeds]
    
    signatures = (zipped
                  .flatMap(lambda x: [[(x[1], i % bands), hashes[i](x[0])] for i, h in enumerate(hashes)])
                  .cache()
                 )
    
    bands = (signatures
             .groupByKey()
             .map(lambda x: [(x[0][1], hash(frozenset(x[1].data))), x[0][0]])
             .groupByKey()
             .cache()
             )

    if c > 0:
      bands = (bands
               .filter(lambda x: len(x[1]) > cluster)
               .cache()
              )

    vector_bucket = (bands
                     .map(lambda x: frozenset(sorted(x[1])))
                     .distinct() 
                     .zipWithIndex()
                     .flatMap(lambda x: map(lambda y: (np.long(y), x[1]), x[0]))
                     .cache()
                    )
    
    bucket_vector = (vector_bucket
                     .map(lambda x: (x[1], x[0]))
                     .cache()
                    )
                  
    buckets = (zipped
               .map(lambda x: (x[1], x[0]))
               .join(vector_bucket)
               .map(lambda x: (x[1][1], x[1][0]))
               .groupByKey()
               .cache()
              )

    jaccard_point = buckets.map(distance_vector).cache()

    return LSHModel(signatures, bands, vector_bucket, bucket_vector, buckets, jaccard_point)

Overwriting lsh.py


In [0]:
"""
Module to read input data and convert the data points into Sparse Vectors.
Then use the lsh.py module to compute the Locality Sensitive Hashing of the
data points.
"""
import argparse
import numpy as np
import os
import lsh

from pyspark import SparkContext, SparkConf
from pyspark.mllib.linalg import SparseVector

def read_text(sc, path):
    text = sc.textFile(path).collect()
    data = []
    for s in text:
        elements = map(int, s[s.index("[") + 1:s.index("]")].split(","))
        v = SparseVector(65535, elements, np.ones(len(elements)))
        data.append(v)
    return data

if __name__ == "__main__":
  
  # Read the input data.
  data = read_text(sc, "documents.txt")
  p = 65537
  m, n, b, c = 1000, 1000, 25, 2
  model = lsh.run(data, p, m, n, b, c)
  print ('Found %s clusters.' % model.buckets.count())

Py4JJavaError: ignored